In [1]:
import pandas as pd
from functools import reduce
import numpy as np

In [2]:
s_raw = """marathon: Big_Jim 2nd_astronaut RidewiththeFlow nobbixl südpfälzer deflo All_mtn Spoon-man mogg AndySch dino113
xc: Dirty_Track Big_Jim Edith_L. A-Abraxas 2nd_astronaut RidewiththeFlow nobbixl südpfälzer deflo Diddo DeluXer Mr.Hahn rhnordpool All_mtn Tony- PanicMen Spoon-man mogg Mountain_4Life AndySch Schoenpflug stummerwinter Schnerrget Anferd Rubin HazelNoos dino113 Ritzelabzieher Daleo
gravel: MikeyBaloooooza A-Abraxas RidewiththeFlow südpfälzer discordius jalgrattad Bene2405 Tony- TiCiLA AndySch stummerwinter shibboleth dino113 Daleo
enduro: Suicide loeffel MikeyBaloooooza mad_raven Duefi cxfahrer mäxx__ Walkerk Tortomat EagleEye Drahtacus Preme Ghostrider72 x_FreiRider_x Lambutz DeluXer Mr.Hahn boescha All_mtn Bene2405 pointidani ElDiabolo666 Tony- ChaosSpeed31 Schoenpflug luisuet1 Schnerrget Anferd shibboleth nanananaMADMAN k0p3 SwishSwushPow bikerniki
gravity: loeffel MikeyBaloooooza mad_raven mäxx__ EagleEye Votec_Tox Drahtacus x_FreiRider_x Lambutz Zask06 Bene2405 ChaosSpeed31 k0p3 SwishSwushPow
dirt: MikeyBaloooooza mad_raven mäxx__ nanananaMADMAN
big_air: Zask06
street: MikeyBaloooooza mad_raven mäxx__ südpfälzer Mr.Hahn Rubin HazelNoos
rennrad: MikeyBaloooooza Edith_L. A-Abraxas EagleEye südpfälzer discordius boescha AndySch Schoenpflug Rubin HazelNoos
bmx: Bene2405
rolle: MikeyBaloooooza Edith_L. discordius Schoenpflug"""

In [3]:
s_arr = s_raw.lower().split('\n')
s_keys = list(map(lambda x: x.split(': ')[0], s_arr))
s_val = list(map(lambda x: list(filter(bool, x.split(': ')[1].split(" "))), s_arr))

In [4]:
lens = list(map(len, s_val))
N = len(lens)
unq, col = np.unique(np.concatenate(s_val),return_inverse=1)
row = np.repeat(np.arange(N), lens)

In [5]:
out = np.zeros((N,len(unq)),dtype=int)
out[row,col] = 1

In [6]:
df = pd.DataFrame(out, columns=unq, index=s_keys).transpose()

In [7]:
df.corr()

,marathon,xc,gravel,enduro,gravity,dirt,big_air,street,rennrad,bmx,rolle
marathon,1.000000,0.489869,0.145969,-0.437229,-0.261387,-0.126629,-0.061684,-0.038016,-0.001855,-0.061684,-0.126629
xc,0.489869,1.000000,0.097254,-0.600006,-0.533585,-0.258496,-0.125919,0.064067,0.145093,-0.125919,0.008914
gravel,0.145969,0.097254,1.000000,-0.293070,-0.118012,0.010532,-0.071822,0.045008,0.247808,0.235987,0.168505
enduro,-0.437229,-0.600006,-0.293070,1.000000,0.340595,0.241747,-0.143929,0.015654,-0.177489,0.117760,-0.026861
gravity,-0.261387,-0.533585,-0.118012,0.340595,1.000000,0.326479,0.235987,0.167758,-0.057709,0.235987,0.010532
dirt,-0.126629,-0.258496,0.010532,0.241747,0.326479,1.000000,-0.034794,0.527269,0.046047,-0.034794,0.196429
big_air,-0.061684,-0.125919,-0.071822,-0.143929,0.235987,-0.034794,1.000000,-0.047314,-0.061684,-0.016949,-0.034794
street,-0.038016,0.064067,0.045008,0.015654,0.167758,0.527269,-0.047314,1.000000,0.364507,-0.047314,0.111004
rennrad,-0.001855,0.145093,0.247808,-0.177489,-0.057709,0.046047,-0.061684,0.364507,1.000000,-0.061684,0.564076
bmx,-0.061684,-0.125919,0.235987,0.117760,0.235987,-0.034794,-0.016949,-0.047314,-0.061684,1.000000,-0.034794


In [8]:
def cond_prob(df, col_idx):
    col = df.iloc[:, col_idx]
    col_total = col.sum()
    cond_values = df.transpose().mul(col.values).transpose().sum()
    probs = cond_values / col_total
    return probs

In [9]:
cols = df.shape[1]

In [10]:
cond_probs = pd.DataFrame([cond_prob(df, idx) for idx in range(cols)], columns=df.transpose().index, index=df.transpose().index)

In [11]:
cond_probs.mul(100).round(2).astype(np.int)

,marathon,xc,gravel,enduro,gravity,dirt,big_air,street,rennrad,bmx,rolle
marathon,100,100,36,9,0,0,0,9,18,0,0
xc,37,100,27,24,0,0,0,13,24,0,6
gravel,28,57,100,28,14,7,0,14,35,7,14
enduro,3,21,12,100,36,12,0,12,12,3,6
gravity,0,0,14,85,100,21,7,21,14,7,7
dirt,0,0,25,100,75,100,0,75,25,0,25
big_air,0,0,0,0,100,0,100,0,0,0,0
street,14,57,28,57,42,42,0,100,57,0,14
rennrad,18,63,45,36,18,9,0,36,100,0,36
bmx,0,0,100,100,100,0,0,0,0,100,0
